## 测试注意力向量计算

In [3]:
import tensorflow as tf
import numpy as np
import keras.backend as K

Using TensorFlow backend.


In [76]:
def my_dot(x):
    c = [tf.reduce_sum(tf.multiply(x[0][:, inx, :], x[1]), -1, keep_dims=True) for inx in range(3)]
    return tf.concat(c, -1)

In [77]:
def my_dot_v2(x, y):
    """注意力点乘函数，快速版本"""
    res = K.batch_dot(tf.expand_dims(y, -1),x, (1, 2))
    return K.reshape(res, [-1, 3])

In [4]:
def att_dot(x):
    """注意力点乘函数"""
    d_btf, q_bf = x
    res = K.batch_dot(tf.expand_dims(y, -1),x, (1, 2))
    return tf.reshape(res, [-1, 3])

In [12]:
def new_att_dot(x):
     d_btf, q_bf = x
     res = tf.matmul(tf.expand_dims(q_bf, -1), d_btf, adjoint_a=True,adjoint_b=True)
     return tf.reshape(res, [-1, 3])

In [5]:
a = tf.placeholder(tf.float32,shape=(None,3,4))
b = tf.placeholder(tf.float32,shape=(None,4))
true_a = np.arange(24).reshape(2,3,4)
true_b = np.arange(8).reshape(2,4)
print(true_a)
print('-'*20)
print(true_a.shape)
print('-'*20)
print(true_b.shape)
print('-'*20)
print(true_b)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
--------------------
(2, 3, 4)
--------------------
(2, 4)
--------------------
[[0 1 2 3]
 [4 5 6 7]]


In [14]:
d = att_dot([a,b])
d.get_shape()
e = new_att_dot([a,b])
e.get_shape()

TensorShape([Dimension(None), Dimension(3)])

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(d, {a: true_a, b: true_b}))
    print("-"*50)
    print(sess.run(e, {a: true_a, b: true_b}))

[[  14.   38.   62.]
 [ 302.  390.  478.]]
--------------------------------------------------
[[  14.   38.   62.]
 [ 302.  390.  478.]]


In [80]:
x = tf.expand_dims(b,-1)
print(x.get_shape())
y = a
print(y.get_shape())

(?, 4, 1)
(?, 3, 4)


In [81]:
res = K.batch_dot(x,y,(1,2))
res = tf.reshape(res,[-1,3])
res.get_shape()

TensorShape([Dimension(None), Dimension(3)])

In [82]:
d = my_dot_v2(a,b)
d.get_shape()

TensorShape([Dimension(None), Dimension(3)])

In [63]:
np.arange(20,24).dot(np.array([4,5,6,7]))

478

## 测试tensorflow的scan

In [4]:
sentence_ids = tf.Variable([5, 8, 1, 3, 0, 34, 8, 7, 3, 8])
attentions = tf.Variable([5, 9, 1, 3, 0, 34, 9, 7, 3, 9],dtype="float32")
word_id = tf.Variable(8)
word_ids = tf.Variable([8,5,7])
aaa = tf.equal(sentence_ids, word_id)
ccc = tf.where(aaa)
qqq = tf.reduce_sum(tf.gather(attentions,ccc))


def sum_prob_of_word(word_ix, sentence_ixs, sentence_attention_probs):
    word_ixs_in_sentence = tf.where(tf.equal(sentence_ixs, word_ix))
    return tf.reduce_sum(tf.gather(sentence_attention_probs, word_ixs_in_sentence))

test_func = lambda x:sum_prob_of_word(x,sentence_ids,attentions)

ppp = test_func(word_id)

def sum_probs_single_sentence(prev,cur):
    candidate_indices_i, sentence_ixs_t, sentence_attention_probs_t = cur
    result = tf.scan(
        fn=lambda prev,x: sum_prob_of_word(x, sentence_ixs_t, sentence_attention_probs_t),
        elems=[candidate_indices_i],
        initializer=tf.Variable(0.,dtype="float32"))
    return result

zzz = sum_probs_single_sentence(None,[word_ids,sentence_ids,attentions])

def func(prev, cur):
    print(cur.get_shape())
    print(cur)
    print(prev)
    return cur

v = tf.Variable(np.arange(24).reshape(2, 3, 4))
# print(v.get_shape())

bbb = tf.scan(func, elems=v)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(ccc.get_shape())
    print(sess.run(zzz))
    # print(sess.run(bbb))

(3, 4)
Tensor("scan_1/while/TensorArrayReadV3:0", shape=(3, 4), dtype=int32)
Tensor("scan_1/while/Identity_1:0", shape=(3, 4), dtype=int32)


(?, 1)
[ 27.   5.   7.]


In [1]:
import tensorflow as tf
import numpy as np


def sum_prob_of_word(word_ix, sentence_ixs, sentence_attention_probs):
    word_ixs_in_sentence = tf.where(tf.equal(sentence_ixs, word_ix))
    return tf.reduce_sum(tf.gather(sentence_attention_probs, word_ixs_in_sentence))

def sum_probs_single_sentence(prev,cur):
    candidate_indices_i, sentence_ixs_t, sentence_attention_probs_t = cur
    result = tf.scan(
        fn=lambda prev,x: sum_prob_of_word(x, sentence_ixs_t, sentence_attention_probs_t),
        elems=[candidate_indices_i],
        initializer=tf.constant(0.,dtype="float32"))
    return result

def sum_probs_batch(candidate_indices_bt, sentence_ixs_bt, sentence_attention_probs_bt):
    result = tf.scan(
        fn=sum_probs_single_sentence,
        elems=[candidate_indices_bt, sentence_ixs_bt, sentence_attention_probs_bt],
        initializer=tf.Variable([1,2,3],dtype="float32"))
    return result

candidate_idx = tf.Variable([
    [16, 21, 8],
    [13, 19, 26],
    [23, 9, 23]
])

sentence_idx = tf.Variable([
    [16, 21, 23, 16, 8, 9, 21],
    [16, 21, 23, 16, 8, 9, 21],
    [16, 21, 23, 16, 8, 9, 21],
])

attention_idx = tf.Variable([
    [0.3, 0.2, 0.1, 0.3, 0, 0, 0],
    [0.3, 0.2, 0.1, 0.3, 0, 0, 0],
    [0.3, 0.2, 0.1, 0.3, 0, 0, 0]
],dtype="float32")

o = sum_probs_batch(candidate_idx, sentence_idx, attention_idx)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(o))

[[ 0.60000002  0.2         0.        ]
 [ 0.          0.          0.        ]
 [ 0.1         0.          0.1       ]]


In [1]:
import tensorflow as tf
import keras.backend as K

a = [1,2,3,4,5]
K.eval(tf.sequence_mask(a,7,dtype=tf.float32))

Using TensorFlow backend.


array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  0.,  0.]], dtype=float32)

In [2]:
import numpy as np

In [1]:
batch_size = 4
A_size = 5

### 测试获取dynamic_rnn输出的有效位

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn import LSTMCell, GRUCell, MultiRNNCell

i = np.random.rand(1000)

In [2]:
with tf.variable_scope('q_encoder'):
    cell = MultiRNNCell(cells=[GRUCell(2)] * 1)
    x = tf.placeholder(dtype=tf.float32, shape=(2, 5, 100), name="x")
    q_lens = tf.placeholder(dtype=tf.int32,shape=(2))
    outputs, last_states = tf.nn.bidirectional_dynamic_rnn(cell_bw=cell,
                                                           cell_fw=cell,
                                                           dtype="float32",
                                                           sequence_length=q_lens,
                                                           inputs=x,
                                                           swap_memory=True)
    q_enc = tf.gather_nd(outputs, tf.stack([tf.range(q_lens.get_shape()[0]), q_lens - 1], axis=1))
    q_enc_c =  tf.concat(outputs,axis=-1)

In [3]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    a, b, c, d = sess.run([outputs, last_states, q_enc, q_enc_c], feed_dict={
        q_lens: (3, 2),
        x: i.reshape(2, 5, 100)
    })
    print("outputs:{}\nstates:{}\nq_enc:{}\nq_enc_c:{}".format(a, b, c, d))

outputs:(array([[[ 0.0560709 , -0.10033438],
        [ 0.07781161, -0.220449  ],
        [ 0.06069176, -0.23297542],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.02716292, -0.05908116],
        [-0.03233079, -0.02452423],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32), array([[[ 0.08484475,  0.06869046],
        [ 0.07913341,  0.17418024],
        [ 0.11813057, -0.0282253 ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.12557939, -0.38799691],
        [-0.09159085, -0.26809931],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32))
states:((array([[ 0.06069176, -0.23297542],
       [-0.03233079, -0.02452423]], dtype=float32),), (array([[ 0.08484475,  0.06869046],
       [-0.12557939, -0.38799691]], dtype=float32),))
q_enc:[[[ 0.          0.        ]
  [ 0.   

### 测试参数初始化

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell, GRUCell

In [9]:
from tensorflow import variable_scope, get_variable

with variable_scope("cc",initializer=tf.orthogonal_initializer()):
    with variable_scope("ddd"):
        a = get_variable("weight",shape=(3,3),dtype=tf.float32)

with variable_scope("cccc",initializer=tf.zeros_initializer()):
    with variable_scope("ccc"):
        b = get_variable("weight",shape=(3,3),dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))
    print(sess.run(b))

[[ 0.89980626  0.31822035  0.29847053]
 [-0.29329553  0.94766068 -0.12616226]
 [-0.3229962   0.02598153  0.94604361]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
